In [ ]:
mygame='PhoenixDeterministic-v4' 

##Installation des dépendances

on change de méthode pour afficher les trajectoires dans colab. 

In [ ]:
!apt-get update
!apt-get install --with-fix-missing -y xvfb x11-utils

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
B

In [ ]:
!pip install pyvirtualdisplay==0.2.* \
             PyOpenGL==3.1.* \
             PyOpenGL-accelerate==3.1.*

In [ ]:
!pip install gym[atari]==0.17.*

In [ ]:
!echo $DISPLAY

In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
!echo $DISPLAY

:1007


Pour tester un agent, on appellera la fonction *record_video*.

In [ ]:
import typing

import gym
import matplotlib.pyplot as plt
import numpy as np
from IPython import display


# represent states as arrays and actions as ints
State = np.ndarray
Action = int

# agent is just a function! 
Agent = typing.Callable[[State], Action]


def uniform_random_policy(state: State,
                          number_actions: int,
                          random_state: np.random.RandomState) -> Action:
    """Select an action at random from the set of feasible actions."""
    feasible_actions = np.arange(number_actions)
    probs = np.ones(number_actions) / number_actions
    action = random_state.choice(feasible_actions, p=probs)
    return action


def make_random_agent(number_actions: int,
                      random_state: np.random.RandomState = None) -> Agent:
    """Factory for creating an Agent."""
    _random_state = np.random.RandomState() if random_state is None else random_state
    return lambda state: uniform_random_policy(state, number_actions, _random_state)

# create the Gym environment
bo = gym.make(mygame)
# create an agent
random_agent = make_random_agent(bo.action_space.n, random_state=None)
bo.close()

In [ ]:
#!nvidia-smi 

## Prétraitement des images / de l'environnement

On utilisera un réseau de neurones à convolutions pour lire directement les images fournies par un émulateur de console ATARI 2600.

Toutefois, on applique quelque transformations aux images reçues (suivant Mnih et al. 2015)
1. chaque action est appliquée 4 fois de suite (pour simuler un temps de réaction humain). On somme les récompenses des 4 états intermédiaires, et on renvoie une seule image qui est obtenue en appliquant la fonction sur chaque pixel (à partir des 4 imaegs intermédiaires) 
2. on redimenssionne l'image rectangulaire en un carré de 84x84 pixels. On transformera les couleurs en rééls compris entre 0 et 1.
3. chaque image est concaténée aux 3 précédentes pour que le réseau ait accès à une notion de mouvement des objets sur l'écran.

In [ ]:
# Taken from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/wrappers.py

import cv2
import numpy as np
import collections

class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 210 * 160 * 3:
            img = np.reshape(frame, [210, 160, 3]).astype(np.float32)
        elif frame.size == 250 * 160 * 3:
            img = np.reshape(frame, [250, 160, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], 
                                old_shape[0], old_shape[1]), dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class Uint8Frame(gym.ObservationWrapper):
    def observation(self, obs):
        return (np.array(obs,dtype=np.uint8))

def make_env(env_name):
    env = gym.make(env_name)
    #env = MaxAndSkipEnv(env)
    #env = FireResetEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    return Uint8Frame(env)

In [ ]:
#import pour les réseaux de neurones
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package

#device = torch.device("cuda")
device = torch.device("cpu")

#pour les logs
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [ ]:
# Taken from 
# https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On/blob/master/Chapter06/lib/dqn_model.py

import numpy as np

class NetworkDQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(NetworkDQN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size(0), -1)
        return self.fc(conv_out)

In [ ]:
test_env = make_env(mygame)
print(test_env.observation_space.shape, test_env.action_space.n)
print(test_env.unwrapped.get_action_meanings())


(4, 84, 84) 8
['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'DOWN', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE']


La fonction record_video prend:
1. un agent qui définit une méthode *\_\_call\_\_* qui effectue une action
2. le nom du répertoire où écrire les fichiers
3. un timeout (en secondes) pour les agents qui ne font rien.


In [ ]:
import time


def record_video(current_agent, dirname, timeout=120):

  FPS=25 #ou 30 ou 60 ??
   
  _display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
  _ = _display.start()

  record_folder= dirname
  visualize=True

  env = make_env(mygame)
  if record_folder:
    env = gym.wrappers.Monitor(env, record_folder, force=True)

  state = env.reset()
  total_reward = 0.0


  steps = 0
  while True:
        if steps > timeout * FPS:
          break
        steps +=1
        start_ts = time.time()
        if visualize:
            env.render()
        
        action = current_agent(state)

        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
        if visualize:
            delta = 1/FPS - (time.time() - start_ts)
            if delta > 0:
                time.sleep(delta)
  print("Total reward: %.2f" % total_reward)

  if record_folder:
        env.close()

In [ ]:
record_video(random_agent, "video_random")

Total reward: 420.00


#Version *classique*




In [ ]:
class QLearningApproxNN1(nn.Module):
  def __init__(self, env, device):
    super(self.__class__, self).__init__()
    
    self.device = device
    self.net = NetworkDQN(env.observation_space.shape, env.action_space.n).to(device)

  def reset(self, env):
    env.reset()

  def get_epsilon_greedy_action(self, env, state, epsilon):
    if torch.rand((1,)).item() < epsilon:
      return torch.randint(0,env.action_space.n,(1,)).item()
    else:
      return torch.argmax(self.net(state)).item()

  def compute_policy(self, env, maxit, epsilon_start=0.8, epsilon_end=0.05, gamma=0.99, alpha=0.001):
    tot_rewards = 0.0
    losses = []

    self.net.train()
    optimizer = torch.optim.Adam(self.net.parameters(), lr=alpha)

    for m in range(maxit):
      done = False
      state = env.reset()
      tot_reward = 0.0
      
      epsilon = epsilon_start - m * (epsilon_start - epsilon_end)  /maxit

      while not done:
        #choix de l'action
        tstate = torch.tensor([state], dtype=torch.float, device=self.device)
        action = self.get_epsilon_greedy_action(env, tstate, epsilon)

        ostate, reward, done, _ = env.step(action)
        tot_reward += reward

        #calcul de l'erreur
        if done:
          estimate = reward
        else:
          next_state = torch.tensor([ostate], dtype=torch.float, device=self.device)
          estimate = (reward + gamma*torch.max(self.net.forward(next_state))).detach()

        error = torch.square(estimate - self.net.forward(tstate)[0][action])

        #calcul du gradient de l'erreur, descente de gradient, remise à zéro des gradients 
        error.backward()
        optimizer.step()
        optimizer.zero_grad()

        state = ostate

      tot_rewards += tot_reward

      if (m+1)%10 == 0:
        avgr = tot_rewards/m
        print(m+1, avgr, epsilon) #, self.parameters)
      
    self.net.eval()
      
  #implémente la politique gloutonne
  def __call__(self, obs):
    state = torch.tensor(obs, dtype=torch.float, device=self.device)
    return torch.argmax(self.net.forward(state)).item()
  


In [ ]:
agent = QLearningApproxNN1(test_env, device)

In [ ]:
#agent.compute_policy(test_env, 100)

In [ ]:
#record_video(agent, "old")

#Tampon d'exemples

On implémente un tampon qui va stocker des nouveaux exemples (via la fonction *push*) et permetre d'en lire aléatoirement (via la fonction *sample*):

In [ ]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a SARSA transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = args
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def reset(self):
        self.memory = []
        self.position = 0

## DQN avec tampon d'exemples et réseau cible

Dans cette partie, on va implémenter le DQN:
- réseau cible. 
  - L'estimation Bellman est calculée par un réseau cible. Ce réseau est initialisé par copie du réseau à apprendre.
  - Puis toutes les *target_update* mises à jour du réseau à apprendre, on le recopie dans le réseau cible.
- tampon d'exemples
  - l'apprentissage génère des transitions de trajectoires qui sont stockées dans un tampon (classe *ReplayMemory*).
  - Pour mettre à jour le réseau, on tire aléatoirement des exemples depuis le tampon (pas directement des trajectoires)
  - Avant de commencer à apprendre, on remplit le tampon (avec *burnin* transitions)

- autres
  - la mise à jour des paramètres du réseau se fait par une descente de gradients avec des lots de taille 32. La fonction d'erreur est la *différence au carré* avec l'estimation. On fait une mise à jour quand on ajouté 4 transitions au tampon.
  - on sauvegarde le modèle toutes les 1000 mises à jour. On utilisera les fonctions *load/save* de pytorch, ainsi que les *state_dict*.
  - pendant l'apprentissage les trajectoires sont générées par une politique $\varepsilon$-gloutonne. La valeur d'$\varepsilon$ diminue linéairement entre *epsilon_start* et *epsilon_end* à chaque mise à jour des paramètres sur les premières 1000000, puis reste constante à *epsilon_end*.



In [ ]:
import random

from google.colab import drive
drive.mount('/content/gdrive')
prefix = "/content/gdrive/My Drive/" 


import random

from google.colab import drive
drive.mount('/content/gdrive')
prefix = "/content/gdrive/My Drive/" 

class QLearningApproxNN(nn.Module):
  def __init__(self, env, device):
    super(self.__class__, self).__init__()
    
    self.device = device
    self.net = NetworkDQN(env.observation_space.shape, env.action_space.n).to(device)

  def reset(self):
    self.net = NetworkDQN(env.observation_space.shape, env.action_space.n).to(self.device)

  def get_epsilon_greedy_action(self, env, state, epsilon):
    if torch.rand((1,)).item() < epsilon:
      return torch.randint(0,env.action_space.n,(1,)).item()
    else:
      return torch.argmax(self.net(state)).item()

  def compute_policy(self, env, maxit, epsilon_start=1.0, epsilon_end=0.05, gamma=0.99, alpha=2.5e-4, 
                     replay_size=700000,
                     update_target = 10000, label=None, save=None):
    tot_rewards = 0.0
    tot_lengths = 0.0

    optimizer = torch.optim.Adam(self.net.parameters(), lr=alpha)
    #optimizer = torch.optim.SGD(self.net.parameters(), lr=alpha)

    target = NetworkDQN(env.observation_space.shape, env.action_space.n).to(device)
    target.load_state_dict(self.net.state_dict())
  
    replay = ReplayMemory(replay_size)

    #nombre de mises à jour
    num_param_updates = 0
    #nombre de transitions générées
    steps = 0
  
    if save:
      checkpoint = torch.load(save)
      self.net.load_state_dict(checkpoint['net_state_dict'])
      target.load_state_dict(checkpoint['target_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      epsilon_start = checkpoint['epsilon_start']
      epsilon_end = checkpoint['epsilon_end']
      num_param_updates = checkpoint['num_param_updates']

    epsilon = epsilon_start

    #on génère d'abord suffisamment d'exemples pour échantillonner "aléatoirement"
    while len(replay) < (replay_size/5):
      state = env.reset()
      done = False
      while not done:
        action = torch.randint(0, env.action_space.n, (1,)).item()
        next_state,reward,done,_ = env.step(action)
        replay.push(state, action, reward, next_state if not done else None)
        state = next_state
        
    #####
    print("let's learn!")

    #Q-Learning commence!

    for m in range(maxit):
      #intialisation pour une nouvelle trajectoire

      state = env.reset()
      tot_reward = 0
      done = False

      c = 0 #nombre de transitions générées pour cette trajectoire
      while not done:
        c += 1
        steps += 1

        #choix d'une action + step + ajout de la transition à replay
        tstate = torch.tensor([state]) / 255.0
        action = self.get_epsilon_greedy_action(env, tstate, epsilon)
        next_state,reward,done,_ = env.step(action)
        replay.push(state, action, reward, next_state if not done else None)
        state = next_state
        tot_reward += reward

        if (steps+1) % 4 == 0:

          batch_size = 32

          #creation d'un batch de taille 32

          states,actions,rewards,next_states = zip(*replay.sample(batch_size))
          state_values = self.net(torch.tensor(states) / 255.0)
          actions = torch.tensor(actions)
          state_action_values = state_values.gather(1, actions.unsqueeze(1)).squeeze()

          rewards = torch.tensor(rewards)

          next_non_final_states = torch.tensor([ns for ns in next_states if ns is not None]) / 255.0
          non_finals = torch.tensor([s is not None for s in next_states], dtype = torch.bool)
          
          next_states_values = torch.zeros_like(state_action_values)
          next_states_values[non_finals] = target(next_non_final_states).max(1)[0].detach()

          expected_gain = rewards + gamma * next_states_values

          #calcul de l'erreur moyenne pour ce batch
          error = torch.square(expected_gain - state_action_values).mean()

          #mise à jour par descente de gradient (via optimizer) + RaZ des gradients 
          error.backward()
          optimizer.step()
          optimizer.zero_grad()

          num_param_updates += 1
          epsilon = max(epsilon_end, epsilon_start - num_param_updates * (epsilon_start - epsilon_end)  /1000000)


          #MaJ périodique du réseau cible par copie
          if num_param_updates % update_target == 0:
            target.load_state_dict(self.net.state_dict())

          #sauvegarde périodique du modèle
          if ((num_param_updates+1)%5000 == 0)  or ((m+1 == maxit) and done):
            print(F"{num_param_updates+1} updates")
            torch.save({
              'net_state_dict': self.net.state_dict(),
              'target_state_dict': target.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'epsilon_start': epsilon_start,
              'epsilon_end': epsilon_end,
              'num_param_updates': num_param_updates,
              
            }, prefix + (("save" if label is None else F"save_{label}") + F"_{num_param_updates+1}.pt"))

        state = next_state

      #la trajectoire est finie
      tot_rewards += tot_reward
      tot_lengths += c

      avgr = tot_rewards/(m+1)
      avgl = tot_lengths/(m+1)
      print(m+1, avgr, avgl, epsilon)


  #politique gloutonne
  def __call__(self, obs):
    state = torch.tensor([obs], dtype=torch.float, device=self.device) / 255.0
    return torch.argmax(self.net(state)).item()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
agent1 = QLearningApproxNN(test_env, device)
agent1.compute_policy(test_env, 10000, epsilon_start=1.0, epsilon_end=0.05, label="1")

In [ ]:
cp = torch.load(prefix+"save_3_675000.pt", device)
agent1 = QLearningApproxNN(test_env, device)
agent1.net.load_state_dict(cp['net_state_dict'])

<All keys matched successfully>

In [ ]:
record_video(agent1,"video675k")

In [ ]:
agent1 = QLearningApproxNN(test_env, device)
agent1.compute_policy(test_env, 1000, label="3", save=prefix+"save_3_1495000.pt", replay_size=100000)